In [1]:
from typing import List
from Dataset.CustomDataset import AgeGroupAndAgeDataset, StandardDataset, AgeDatasetKL
from Dataset.CustomDataLoaders import CustomDataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from Utils import AAR, CSVUtils, AgeConversion
from Utils.Validator import Validator

c:\Users\Gio\.conda\envs\AV_project\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Caricamento del dataframe
df = CSVUtils.get_df_from_csv("./training_caip_contest.csv", "./training_caip_contest/")

In [3]:
df_train, df_val = train_test_split(df, test_size=0.25, random_state=42)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
#########################
import pandas as pd
df = CSVUtils.get_df_from_csv("./training_caip_contest.csv", "./training_caip_contest/")
df_train, df_val = train_test_split(df, test_size=0.25, random_state=42)
aug = CSVUtils.get_df_from_csv("./augumentation_two_age.csv", "./newAugmentationDataset/")
df_train = pd.concat([df_train, aug], ignore_index=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
#########################

from torchvision import transforms
import torch

transform_func = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(size=224),
    transforms.RandomHorizontalFlip(),
    transforms.RandAugment(2, 9),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

transform_func_val = transforms.Compose([
    transforms.Resize(224),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

# Implementazione di un Dataset utilizzando "CustomDataset" per l'architettura con Film
cd_train = AgeDatasetKL(df_train, path_col="path", label_col="age", label_function="Linear", 
                        transform_func=transform_func)
cd_train.set_n_classes(81)
cd_train.set_starting_class(1)
dm_train = CustomDataLoader(cd_train)
# dl_train = dm_train.get_unbalanced_dataloader(batch_size=64, shuffle=True, drop_last=True, num_workers=16, prefetch_factor=4, pin_memory=True)
dl_train, sampler = dm_train.get_balanced_class_dataloader2(class_ranges=[(0, 11), (11, 21), (21, 31), (31, 41), (41, 51), (51, 61), (61, 71), (71, 91)], 
                                                            batch_size=64, num_workers=12, prefetch_factor=4, pin_memory=True)
sampler.n_batches = 2000
cd_val = StandardDataset(df_val, path_col="path", label_col="age", label_function="CAE", transform_func=transform_func_val)
cd_val.set_n_classes(81)
cd_val.set_starting_class(1)
validator = Validator(cd_val, AgeConversion.EVAge, 64, num_workers=8, prefetch_factor=4)

In [4]:
from ResNetFilmed.resnet import ResNetFiLMed, BackBone, ResNetNotFiLMed, DoNothingLayer
from torchvision.models import resnet18, ResNet18_Weights, efficientnet_b0, EfficientNet_B0_Weights
import torch
from torch import optim
import torch.nn.functional as F
from torch import nn

####################################################
EPOCHS = 24
####################################################

backbone = resnet18(ResNet18_Weights.IMAGENET1K_V1)
backbone.fc = DoNothingLayer()
backbone.train()
backbone.requires_grad_(True)
backbone.to("cuda")
model_age = ResNetNotFiLMed(backbone, 81)
opt = optim.SGD(set([*backbone.parameters(), *model_age.fc0.parameters()]), lr=0.1, weight_decay=5e-4)
scheduler = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(dl_train), epochs=EPOCHS, three_phase=True)
kl = nn.KLDivLoss(reduction="batchmean")

c:\Users\Gio\.conda\envs\AV_project\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [5]:
best_val_aar = -1
import numpy as np

for e in range(EPOCHS):
    with tqdm(dl_train, unit=" batch") as tepoch:
        for batch in tepoch:
            opt.zero_grad()
            x, y = batch
            x = x.to("cuda")
            y_age: torch.Tensor = y[0].to("cuda")
            y_age_kl: torch.Tensor = y[1].to("cuda")

            out_age = model_age(x)
            loss_age_kl: torch.Tensor = kl(F.log_softmax(out_age, dim=-1), y_age_kl)

            out = F.softmax(out_age, dim=-1)
            out = AgeConversion.EVAge(out).to("cuda")
            loss_age = torch.mean(torch.abs(y_age - out))

            loss = loss_age_kl + loss_age
            loss.backward()
            opt.step()
            scheduler.step()

            tepoch.set_postfix(loss_age_kl=loss_age_kl.detach().cpu().numpy(), loss_age=loss_age.detach().cpu().numpy())

    def forward_function(x):
        out = model_age(x)
        out = F.softmax(out, dim=-1)
        return out

    ae, mae, val_aar, val_aar_old = validator.validate_ext2(forward_function)
    print(ae)
    print(mae, val_aar, val_aar_old)

    aes = np.array([float(x) for x in list(ae.values())])
    sampler.p = aes/aes.sum()

    if best_val_aar < val_aar:
        best_val_aar = val_aar
        torch.save(model_age.state_dict(), "./model_age_feature_simple.pt")
        print("Saved model")

100%|██████████| 2247/2247 [01:38<00:00, 22.77 batch/s]


{0: tensor(7.1873, dtype=torch.float64), 1: tensor(7.1145, dtype=torch.float64), 2: tensor(8.8504, dtype=torch.float64), 3: tensor(11.7164, dtype=torch.float64), 4: tensor(12.7811, dtype=torch.float64), 5: tensor(11.2289, dtype=torch.float64), 6: tensor(6.7233, dtype=torch.float64), 7: tensor(4.5071, dtype=torch.float64)}
tensor(10.5586, dtype=torch.float64) tensor(1.7557, dtype=torch.float64) tensor(0., dtype=torch.float64)
Saved model


100%|██████████| 2247/2247 [01:38<00:00, 22.77 batch/s]


{0: tensor(8.8012, dtype=torch.float64), 1: tensor(7.4721, dtype=torch.float64), 2: tensor(9.4692, dtype=torch.float64), 3: tensor(11.4635, dtype=torch.float64), 4: tensor(10.4078, dtype=torch.float64), 5: tensor(7.8305, dtype=torch.float64), 6: tensor(5.7266, dtype=torch.float64), 7: tensor(4.8704, dtype=torch.float64)}
tensor(9.6417, dtype=torch.float64) tensor(2.4820, dtype=torch.float64) tensor(0.4820, dtype=torch.float64)
Saved model


100%|██████████| 2247/2247 [01:38<00:00, 22.86 batch/s]


...

100%|██████████| 2247/2247 [01:39<00:00, 22.54 batch/s]


{0: tensor(6.2008, dtype=torch.float64), 1: tensor(4.1306, dtype=torch.float64), 2: tensor(5.2015, dtype=torch.float64), 3: tensor(6.8863, dtype=torch.float64), 4: tensor(7.4300, dtype=torch.float64), 5: tensor(5.2430, dtype=torch.float64), 6: tensor(4.2950, dtype=torch.float64), 7: tensor(4.3728, dtype=torch.float64)}
tensor(6.0141, dtype=torch.float64) tensor(3.7129, dtype=torch.float64) tensor(2.6988, dtype=torch.float64)


100%|██████████| 2247/2247 [01:39<00:00, 22.56 batch/s]


{0: tensor(7.2949, dtype=torch.float64), 1: tensor(3.7520, dtype=torch.float64), 2: tensor(5.2236, dtype=torch.float64), 3: tensor(6.2767, dtype=torch.float64), 4: tensor(6.1417, dtype=torch.float64), 5: tensor(4.5805, dtype=torch.float64), 6: tensor(3.2703, dtype=torch.float64), 7: tensor(5.1620, dtype=torch.float64)}
tensor(5.4251, dtype=torch.float64) tensor(3.7252, dtype=torch.float64) tensor(3.3001, dtype=torch.float64)


 15%|█▍        | 293/2000 [00:42<02:01, 14.10 batch/s, loss_age=2.9201164, loss_age_kl=1.2730490926565992] 

In [ ]:
####################################################
EPOCHS = 12
####################################################

dl_train = dm_train.get_balanced_class_dataloader(class_ranges=[(0, 11), (11, 21), (21, 31), (31, 41), (41, 51), (51, 61), (61, 71), (71, 91)], 
                                                  batch_size=64, num_workers=12, prefetch_factor=4, pin_memory=True)

model_age.load_state_dict(torch.load("./model_age_feature_simple.pt"))
opt = optim.SGD(set([*model_age.fc0.parameters()]), lr=0.1, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CyclicLR(opt, base_lr=0.1/25, max_lr=0.1, step_size_up=int(len(dl_train)))

In [ ]:
def forward_function(x):
    out = model_age(x)
    out = F.softmax(out, dim=-1)
    return out

ae, mae, val_aar, val_aar_old = validator.validate_ext2(forward_function)
print(ae, mae, val_aar, val_aar_old)

100%|██████████| 4493/4493 [02:21<00:00, 31.72 batch/s]


tensor(2.2409, dtype=torch.float64) tensor(4.9902, dtype=torch.float64) tensor(5.8763, dtype=torch.float64)


In [ ]:
best_val_aar = val_aar

for e in range(EPOCHS):
    with tqdm(dl_train, unit=" batch") as tepoch:
        for batch in tepoch:
            opt.zero_grad()
            x, y = batch
            x = x.to("cuda")
            y_age: torch.Tensor = y[0].to("cuda")
            y_age_kl: torch.Tensor = y[1].to("cuda")

            out_age = model_age(x)
            loss_age_kl: torch.Tensor = kl(F.log_softmax(out_age, dim=-1), y_age_kl)

            out = F.softmax(out_age, dim=-1)
            out = AgeConversion.EVAge(out).to("cuda")
            loss_age = torch.mean(torch.abs(y_age - out))

            loss = loss_age_kl + torch.square(loss_age - mae)
            loss.backward()
            opt.step()
            scheduler.step()

            tepoch.set_postfix(loss_age_kl=loss_age_kl.detach().cpu().numpy(), loss_age=loss_age.detach().cpu().numpy())

    def forward_function(x):
        out = model_age(x)
        out = F.softmax(out, dim=-1)
        return out

    ae, mae, val_aar, val_aar_old = validator.validate_ext2(forward_function)
    print(ae)
    print(mae, val_aar, val_aar_old)

    if val_aar > best_val_aar:
        best_val_aar = val_aar
        torch.save(model_age.state_dict(), "./model_age_classification_simple.pt")
        print("Saved model")

100%|██████████| 4493/4493 [01:58<00:00, 38.06 batch/s]


tensor(4.3330, dtype=torch.float64) tensor(3.5900, dtype=torch.float64)


100%|██████████| 4493/4493 [02:03<00:00, 36.30 batch/s]


tensor(3.4237, dtype=torch.float64) tensor(2.7052, dtype=torch.float64)


100%|██████████| 4493/4493 [02:19<00:00, 32.31 batch/s]


...

100%|██████████| 4493/4493 [01:53<00:00, 39.44 batch/s]


tensor(5.2628, dtype=torch.float64) tensor(4.7096, dtype=torch.float64)


100%|██████████| 4493/4493 [01:51<00:00, 40.46 batch/s]


tensor(5.3480, dtype=torch.float64) tensor(4.7429, dtype=torch.float64)


: 